In [ ]:
# Importing necessary libraries
import pandas as pd
import numpy as np
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_absolute_error
import warnings

## 4. Holt-Winters model

### 4.1.One day ahead forecasting using Holt-Winters Model for the month of APRIL 2021:

we will use this code to check the accuracy of the Holt-Winters model model to compare with(ANN,SARIMA AND SARIMAX) other models .  we are doing this to select the best two models among SARIMA, ANN,SARIMAX AND Holt-Winters model.

In [2]:
# Suppress warnings
warnings.filterwarnings("ignore", message="Optimization failed to converge")


# Loading data
file_path = 'Database_1_capped.csv'  # Update with your file path
print("Loading data...")
data = pd.read_csv(file_path)

# Converting 'Date and Time' to datetime 
data['Date and Time'] = pd.to_datetime(data['Date and Time'], format='%d/%m/%Y %H:%M', errors='coerce')
data.dropna(subset=['Date and Time'], inplace=True)
data.set_index('Date and Time', inplace=True)
data = data.asfreq('30T')  

# Defining target variable
target = 'Demand_Capped'

# Splitting the data into training and testing sets
train_start = '2020-04-01'
train_end = '2021-03-31'
test_start = '2021-04-01'
test_end = '2021-04-30'

train_data = data.loc[train_start:train_end][target]
test_data = data.loc[test_start:test_end][target]

# Initializing storage for forecasts
one_day_forecasts = []

# Forecasting one day at a time
print("Performing one-day-ahead forecasting with Holt-Winters...")
current_date = pd.to_datetime(test_start)
while current_date <= pd.to_datetime(test_end):
    # Defining the forecast range for the next day (48 intervals for one day)
    forecast_range = pd.date_range(current_date, periods=48, freq='30T')

    # Fitting Holt-Winters model to the training data
    try:
        hw_model = ExponentialSmoothing(
            train_data, 
            seasonal='add', 
            seasonal_periods=48  # Daily seasonality for 30-minute intervals
        ).fit()  # No extra parameters
    except ValueError as e:
        print(f"Error fitting Holt-Winters model: {e}")
        break

    # Forecasting for the current day
    daily_forecast = hw_model.forecast(steps=48)
    daily_forecast.index = forecast_range

    # Storing predictions
    one_day_forecasts.append(pd.DataFrame({
        'Date and Time': forecast_range,
        'Forecast': daily_forecast.values
    }))

    # Updating the training data with the actual observations
    train_data = pd.concat([train_data, test_data.loc[forecast_range]])

    # Moving to the next day
    current_date += pd.Timedelta(days=1)

# Combining all forecasts
one_day_forecasts = pd.concat(one_day_forecasts)

# Merging forecasts with actual values for evaluation
results = one_day_forecasts.copy()
results['Actual'] = results['Date and Time'].map(test_data)

# Dropping rows where actual values are missing
results.dropna(subset=['Actual'], inplace=True)

# Calculating evaluation metrics
mad = mean_absolute_error(results['Actual'], results['Forecast'])
mape = np.mean(np.abs((results['Actual'] - results['Forecast']) / results['Actual'])) * 100

# Printing evaluation metrics
print(f"\nMean Absolute Deviation (MAD): {mad:.2f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")

# Saving results
results.to_csv('holt_winters_forecast_results.csv', index=False)

# Printing sample forecasts
print("\nSample One-Day-Ahead Forecasts:")
print(results.head())


Loading data...
Performing one-day-ahead forecasting with Holt-Winters...

Mean Absolute Deviation (MAD): 681.06
Mean Absolute Percentage Error (MAPE): 6.88%

Sample One-Day-Ahead Forecasts:
        Date and Time      Forecast   Actual
0 2021-04-01 00:00:00  10388.328804  10326.0
1 2021-04-01 00:30:00  10089.010864  10029.0
2 2021-04-01 01:00:00   9777.440175   9732.0
3 2021-04-01 01:30:00   9549.727248   9585.5
4 2021-04-01 02:00:00   9346.476810   9439.0


### We got Mean Absolute Deviation (MAD): 681.06 and Mean Absolute Percentage Error (MAPE): 6.88% for  Holt-Winters model.

its not performing well compared to other models so we are excluding this from inclusion in hybrid model.